## 체크보드를 이용한 Calibration용 파일
- 평평한 체크보드가 필요합니다.

In [ ]:
import numpy as np
import cv2

In [ ]:

# 체스판 이미지에서 코너를 검출하기 위한 설정 
# (내부 교차점 가로 개수, 내부 교차점 새로 개수)
CHESSBOARD_SIZE = (8, 5)
CHESSBOARD_FLAGS = cv2.CALIB_CB_ADAPTIVE_THRESH | cv2.CALIB_CB_NORMALIZE_IMAGE

# 카메라 캘리브레이션을 위한 샘플 이미지 경로
CALIBRATION_IMAGES = ['image/captured_img_0.png','image/captured_img_1.png','image/captured_img_2.png','image/captured_img_3.png','image/captured_img_4.png',
                      'image/captured_img_5.png','image/captured_img_6.png','image/captured_img_7.png','image/captured_img_8.png','image/captured_img_9.png',
                      'image/captured_img_10.png','image/captured_img_11.png','image/captured_img_12.png','image/captured_img_13.png','image/captured_img_14.png',
                      'image/captured_img_15.png','image/captured_img_16.png','image/captured_img_17.png','image/captured_img_18.png','image/captured_img_19.png']

# 체스판 코너를 검출하기 위한 객체 생성
chessboard_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# 체스판 코너를 검출하기 위한 함수
def find_chessboard_corners(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, CHESSBOARD_SIZE, None, CHESSBOARD_FLAGS)
    if ret:
        corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), chessboard_criteria)
    return ret, corners

# 캘리브레이션에 필요한 체스판 코너 검출 결과
object_points = []
image_points = []

# 체스판 코너 검출
for image_path in CALIBRATION_IMAGES:
    ret, corners = find_chessboard_corners(image_path)
    if ret:
        object_points.append(np.zeros((CHESSBOARD_SIZE[0] * CHESSBOARD_SIZE[1], 3), np.float32))
        object_points[-1][:, :2] = np.mgrid[0:CHESSBOARD_SIZE[0], 0:CHESSBOARD_SIZE[1]].T.reshape(-1, 2)
        image_points.append(corners)

# 카메라 캘리브레이션
image_size = cv2.imread(CALIBRATION_IMAGES[0]).shape[:2]
ret, camera_matrix, distortion_coefficients, rotation_vectors, translation_vectors = cv2.calibrateCamera(
    object_points, image_points, image_size, None, None
)

# 결과 출력
print("Camera matrix:\n", camera_matrix)
print("Distortion coefficients:\n", distortion_coefficients)
np.save("intrinsic_parameters/camera_matrix.npy",camera_matrix)
np.save("intrinsic_parameters/distortion_coefficients.npy",distortion_coefficients)
